# Mobile robots project

<div style="border: 2px solid #4CAF50; border-radius: 10px; padding: 15px; background-color: #E8F5E9; color: #333;">
    <strong>T30 Members:</strong>
    <ul>
        <li>Jiwon You</li>
        <li>Adriana Nancy Orellana Torrico</li>
        <li>Charles Froessel</li>
        <li>Johann Lugon-Moulin</li>
</div>



-------

# Project description

## 1. Environment

- A good feature needs to be easy to extract and easy to match, robust to change in illumination, camera viewpoint and highly distinctive.
- From 3D to 2D: perspective projection
- Gaussian smoothing: removes high-frequency noise from the image with a gaussian kernel G
- Edge detection: canny edge filter (combination of smoothing + edge detector into one operator) is an optimal detector, to reduce its cost we can numerically approximate its behavior computing the edge direction and magnitude with Sobel.
- Sobel filter: from a row image convert to gray scale, then filter the image with gaussian or sobel filter, apply thresholding to get the edges, and finally non-maximum suppression to get the final edges.


## 2. Components

*TODO:* For each section, please explain how your imlementation works and how you have tested it. You can add diagrams, images or videos to make it easier to understand and show that your components are working as expected. Don't forget to cite external sources if you used any (even course materials).

### 2.1. Computer vision and image processing

**Responsible:** Adriana Nancy Orellana Torrico

**Slide intructions:** Your environment has to contain a set of obstacles that the Thymio avoids through global navigation. That is to say, the Thymio should avoid these obstacles without using the sensors to detect them. (remove when completed)

### 2.2. Path planning

**Responsible:** Charles Froessel

**Slides intructions:** The objective is that the Thymio goes from an arbitrary position in the map to a target that can be placed anywhere in the environment. These will be changed during the demo to see how your system performs. (remove when completed) 

**Link for schematics (Use epfl account):** https://epflch-my.sharepoint.com/:p:/r/personal/charles_froessel_epfl_ch/_layouts/15/doc.aspx?sourcedoc=%7B20d9373d-95f6-4339-9879-352d763cab56%7D&action=edit 

### 2.3. Pose estimation

**Responsible:** Jiwon You

#### 2.3.1 System Identification of a differetial drive robot
Below are the names to denote state/control variables of Thymio. \
\
$u_l$ = left motor control input, `motor.left.target` \
$u_r$ = right motor control input, `motor.right.target` \
$u_{l,m}$ = measured left motor speed, `motor.left.speed` \
$u_{r,m}$ = measured left motor speed, `motor.right.speed` \
$v_{l}$ = linear velocity of left wheel \
$v_{r}$ = linear velocity of right wheel \
$v$ = linear velocity of Thymio \
$w$ = angular velocity of Thymio 

First, in `calibration.ipynb` I ran calibration code from exercise 8 to discover relationship between motor speed and linear velocity of Thymio. We will mainly use $u_l$ and $u_r$ around 100 to control Thymio, so we investigate linear velocity and motor speed variance at $u=100$. Moreover, we assume that conversion between motor speed and linear velocity is linear around this point. \
![My Image](figure/ground_sensor.png) \

Knowing that there is a thin white stripe between black blocks every 5cm, we can detect peaks from ground sensor and compute linear speed from the time it took to cross 6 blocks.\

It returned linear velocity = `3.26cm/s` for motor speed 100, and thus conversion factor from motor speed to linear velocity was `0.0326`. \

![My Image](figure/target_and_measured_speed_of_motor.png)\

In the histogram below, measured motor speed forms a Gaussian distribution around motor target speed of 100.\

![My Image](figure/measured_motor_speed_histogram.png)

With $\alpha$ as conversion factor and $L$ as Thymio wheel axle length,  \
$v_l = \alpha u_l$ \
$v_r = \alpha u_r$ \
$v = \frac{v_l + v_r}{2}$ \
$w = \frac{v_r - v_l}{L}$ \
Conversion between $v,w$ and $u_l, u_r$ is implemented in `utils.py`.
$$
\begin{align*}
\begin{bmatrix}
v \\
w
\end{bmatrix}
&= TuningMatrix \cdot
\begin{bmatrix}
\frac{\alpha}{2} & \frac{\alpha}{2} \\
\frac{-\alpha}{L} & \frac{\alpha}{L} 
\end{bmatrix} \cdot
\begin{bmatrix}
u_l \\
u_r
\end{bmatrix} = A\cdot u 
\end{align*}
$$ //
```py
    wheel_axle_length = 9.5 # in cm
    thymio_speed_to_cms = 0.03260
    A = thymio_speed_to_cms* np.array([[0.5, 0.5],[-1/wheel_axle_length, 1/wheel_axle_length]]) # theoretical kinematics model, but is understimating
    tuning_matrix = np.array([[1.65,1.65],[2,2]])# compensate for underestimation
    A = np.multiply(tuning_matrix, A)

    def from_u_to_vw(ul, ur):
        vw = A@np.array([ul, ur]) 
        return vw[0], vw[1] # returns v, w in cm/s, rad/s

    def from_vw_to_u(v,w):
        A_inv = np.linalg.inv(A)
        ulur = A_inv @ np.array([v,w]) 
        return int(ulur[0]), int(ulur[1]) # returns ul, ur as int
```
Note that in reality, theoretical conversion model was underestimating actual v and w. For a quick fix, I added empirically tuned `tuning_matrix` to compensate for underestimation. 

Let $X = (x,y,\theta, v, w)^{T}$ denote Thymio state. Discrete time motion model of Thymio is as follows. In particular, we use $v_{i}$ and $w_{i}$ as a temporary prediction of $v_{i+1}$ and $w_{i+1}$. This is reasonable because in our control policy Thymio is moving at constant speed in each control phase(`path_following`, `local_avoidance`, `get_back_to_path`, which will be further explained in motion control section). 
$$
\begin{align*}
X_{i+1} 
&= f(X_i, u_i) \\
&=
\begin{bmatrix}
x_i + v_i\cdot cos(\theta_i)\Delta t\\
y_i + v_i\cdot sin(\theta_i)\Delta t\\
\theta_i + w_i\cdot\Delta t \\
v_{i}\\
w_{i}
\end{bmatrix} \\
\end{align*}
$$ 
This is implemented in `ExtendedKalmanFilter.move(u)` of `kalman.py`, which is  the state prediction step.\
```py
    def move(self, u):
        assert(u.shape[0] == 2) # u is np.array containing v_,w_
        x,y,theta,v,w = self.X
        # v_, w_ = u
        self.X[0] = x + v*np.cos(theta)*self.dt 
        self.X[1] = y + v*np.sin(theta)*self.dt
        self.X[2] = self.clip_theta(theta + w * self.dt)
        self.X[3] = v 
        self.X[4] = w
``` 

#### 2.3.2 Process and measurement noise
For Kalman Filter, let Q denote process noise matrix, R measurement noise matrix, $R_c$ camera noise matrix, $R_{vw}$ linear/angular velocity measurement noise matrix. Since we can't really estimate process noise, I used empirical tuning for Q. 
$$
R = 
\begin{bmatrix}
R_c & 0 \\
0 & R_{vw}
\end{bmatrix}
$$
Measurement noise is split into camera noise and vw noise. It is safe to assume that camera noise and vw noise are uncorrelated, and thus R is in block diagonal form. \

Camera measurements are faily accurate, and we can use a small value for camera noise. However, when aruco marker on Thymio is blocked, camera cannot detect Thymio. For such case, camera will return $(x_{cam}, y_{cam}, \theta_{cam}) = (0, 0, 0)$ and camera noise will be set to infinity, which excludes camera measurement from updating Thymio state. This is implemented in `ExtendedKalmanFilter.switch_mode()` of `kalman.py`.\
```py
    NOISE_COV_CAMERA = 0.0001*np.eye(3)
    NOISE_COV_CAMERA_BLOCKED=9999999*np.eye(3)
    def switch_mode(self, camera_blocked):
        if camera_blocked and self.camera_available:
            self.R[0:3, 0:3] = NOISE_COV_CAMERA_BLOCKED
            self.camera_available = False
        elif not camera_blocked and not self.camera_available:
            self.R[0:3,0:3] = NOISE_COV_CAMERA
            self.camera_available = True
``` 
We have previously seen that motor speed measurement noise resembles Gaussian. Since we have direct access to motor speed measurement, we can easily compute motor speed variance. We assume that noise from left and right motor are uncorrelated.\
![My Image](figure/target_and_measured_speed_of_motor.png)\

Then using linear transformation from `utils.py`, we can convert motor speed variance to linear/angular velocity variance. As in 2.3.1, $\alpha$ denotes Thymio speed conversion factor.
$$
\begin{align*}
\begin{bmatrix}
v \\
w
\end{bmatrix}
&= TuningMatrix \cdot
\begin{bmatrix}
\frac{\alpha}{2} & \frac{\alpha}{2} \\
\frac{-\alpha}{L} & \frac{\alpha}{L} 
\end{bmatrix} \cdot
\begin{bmatrix}
u_l \\
u_r
\end{bmatrix} = A\cdot u \\
\Sigma_{u} &= 
\begin{bmatrix}
\sigma_{ul}^2 & 0 \\
0 & \sigma_{ur}^2
\end{bmatrix} \\
\Sigma_{vw} &= A\Sigma_{u} A^T
\end{align*}
$$ //
```py
    from utils import A
    var_l_speed = np.var(l_speed[idx:]) # took from idx to avoid the initial transient and devid by the conversion factor
    var_r_speed = np.var(r_speed[idx:])
    cov_vw = A @ np.array([[var_l_speed,0],[0, var_r_speed]]) @ A.T
```
$$
R_{vw} =
\begin{bmatrix}
0.0227 & -0.0022 \\
-0.0022 & 0.0014
\end{bmatrix}
$$
Note that $R_{vw}$ is not diagonal. This agrees with our intuition that linear/angular velocity noise are correlated.
#### 2.3.3 Extended Kalman Filter implementation
Since our Thymio is a time-variant nonlinear system and noises can be assumed as Gaussian, we are using Extended Kalman Filter.
$$
\begin{align*}
X_{i+1} 
&= f(X_i, u_i) \\
&=
\begin{bmatrix}
x_i + v_i\cdot cos(\theta_i)\Delta t\\
y_i + v_i\cdot sin(\theta_i)\Delta t\\
\theta_i + w_i\cdot\Delta t \\
v_{i}\\
w_{i}
\end{bmatrix} \\
y &= h(X) = X
\end{align*}
$$ 
First we compute Jacobian of $f(X_i, u_i)$ to linearize our system.
$$
\begin{align*}
F
&= \frac{\partial f(X,u)}{\partial X} \bigg|_{X_i, u_i}
&=
\begin{bmatrix}
1 & 0 & -sin(\theta)v\Delta t & cos(\theta)\Delta t & 0 \\
0 & 1 & cos(\theta)v\Delta t & sin(\theta)\Delta t & 0 \\
0 & 0 & 1 & 0 & \Delta t \\
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 1
\end{bmatrix} 
\end{align*}
$$
Predict state covariance
$$
P = FPF^{T}+Q
$$
Predict state
$$
\bar X = f(X,u)
$$
Measurement residual. For our case, measurement funtion $h(X)$ is an identity function that returns $X$, and thus Jacobian of $h(X)$ is an identity matrix.
$$
y = z - h(X) = z - X
$$
$$
H = \frac{\partial h(X)}{\partial X} = 
\begin{bmatrix}
1 & 0 & 0 & 0 & 0\\
0 & 1 & 0 & 0 & 0\\
0 & 0 & 1 & 0 & 0\\
0 & 0 & 0 & 1 & 0\\
0 & 0 & 0 & 0 & 1\\
\end{bmatrix}
$$
Residual covariance
$$
S = HPH^{T}+R
$$
Near optimal Kalman gain
$$
K = PH^{T}S^{-1}
$$
update state
$$
X = X + Ky
$$
update state covariance
$$
P = (I-KH)P
$$
This is implemented in `ExtendedKalmanFilter.predict_and_update()` of `kalman.py`.
```py
    def predict_and_update(self, u, z):
        assert(u.shape[0] == self.control_dim)
        assert(z.shape[0] == self.measurement_dim)
        # prediction step
        F = self.compute_F()
        self.move(u)
        self.P = F @ self.P @ F.T + self.Q

        #update step
        y = z - self.X # use measurement model H(X) = X
        y[2] = self.clip_theta(y[2]) # clip theta to prevent overflow
        S = self.H @ self.P @ self.H.T + self.R
        S_inv = np.linalg.inv(S)
        K = self.P @ self.H.T @ S_inv
        self.X = self.X + K @ y
        self.P = (np.eye(self.state_dim) - K @ self.H) @ self.P
        self.state_trajectory.append(self.X)
``` 
Consider a case where $\theta_{i+1} = 0.01, \theta_i = 2\pi - 0.01$. Simple subtraction would yield 
$$ 
\Delta\theta = \theta_{i+1} - \theta_i = 0.02 - 2\pi
$$
, but the value should be just 0.02.

To prevent such overflow, we use `ExtendedKalmanFilter.clip_theta()` to keep $\theta$ values in the range of $[-\pi, \pi]$.
```py
    def clip_theta(self, theta):
        return (theta + np.pi) % (2 * np.pi) - np.pi
``` 
In `kalman.ipynb` I tested my implementation with a simple simulation under noise. Despite initial state with large error, EKF is able to converge to true state.
![My Image](figure/ekf_simulation.png)
#### References
Control your Thymio in Python, Basics of Mobile Robotics \
Solution 8, Basics of Mobile Robotics \
https://en.wikipedia.org/wiki/Extended_Kalman_filter \
https://github.com/rlabbe/filterpy 

##### Acknowledgement
overall structure of ExtendedKalmanFilter class was inspired by https://github.com/grilloandrea6/mobile-robotics-project 

### 2.4. Motion Control

**schematic of how motion control work**

<div style="text-align: center;">
    <img src="figure/motion_control.jpg" alt="motioncontr" style="width:70%;"/>
</div>

**fonction : find_how_go to next target**
<br>

When we are in the path following, The thymio turn to the direction of the next point and then go straight to it. The global path to avoid the object on the ground is made of straight lines. If we make curves, it can happend that we go on this object because we dont' recompute the path every loop. So to avoid this situation, turn and then go straight is the best way to to do.

code:
```py
    dx = x_goal - x
    dy = y_goal - y

    # Conversion in polar coordinates
    alpha = np.arctan2(dy, dx) - theta  # angle relative to the target
    alpha = (alpha + np.pi)%(2*np.pi) - np.pi

    desired_theta = np.arctan2(dy, dx)
    dtheta = (desired_theta - theta + np.pi)% (2*np.pi) - np.pi
    if abs(dtheta) > 3*np.pi / 180: # abs(dtheta) larger than 3 degree
        v_ = 0
        w_ = np.sign(dtheta)*0.314 #turn on himself
    else:
        v_ = 3.3 #go straight
        w_ = 0
    return v_, w_
```

### 2.5. Obstacle avoidance

**Responsible:** Johann Lugon-Moulin

We have decided that for the local avoidance we use only the IR sensors to detecte the object and for avoiding the obstacle too. Our Robot is like blinded to bes sur that he can handled every obstacle even if the camera is down.

**Step1** 
<br> 

Thymio take the new path and go to the next_target in the dessin the goal 

<div style="text-align: center;">
    <img src="figure/localnav_step1.png" alt="Step1" style="width:70%;"/>
</div>


**Step 2** 
<br> 

Obstacle detected with IR sensor . Change mode to « local_avoidance » and then check if sensors left have higher value than right. Update X_entrance and Y-entrance and angle at entrance
$$
\text{Sensors left} > \text{Sensors right} \implies \text{wall\_on} = \text{left}
$$

$$
\text{Sensors right} > \text{Sensors left} \implies \text{wall\_on} = \text{right}
$$

<br>
<div style="text-align: center;">
    <img src="figure/localnav_step2.png" alt="Step2" style="width:70%;"/>
</div>
<br>
Code:
<br>

```py
   if self.control_mode == "path_following":
            activate_local_avoidance = local_avoidance.check_obstacle(prox_horizontal)
            if activate_local_avoidance:
                self.control_mode = "local_avoidance"
                self.local_nav.initialize(prox_horizontal)
                print("local avoidance activated")
                self.x_entrance = x
                self.y_entrance = y
                self.alpha_entrance = theta
``` 
```py
        def initialize(self, prox_horizontal):
        if prox_horizontal[0]+ prox_horizontal[1]<=prox_horizontal[3] + prox_horizontal[4]:
            self.wall_on = "right"
            self.mode = "turning"
            print(f"local nav mode = {self.mode}, wall on {self.wall_on}")
        else:
            self.wall_on = "left"
            self.mode = "turning"
            print(f"local nav mode = {self.mode}, wall on {self.wall_on}")
```

**Step 3**
<br>

Thymio turn ( right or left like we check before) on himself until he don’t see the object. Then the local avoidance mode became wall_following  
<br>
<div style="text-align: center;">
    <img src="figure/localnav_step3.png" alt="Step3" style="width:70%;"/>
</div>
<br>
code:
<br>

```py
    if self.mode == "turning":
            if self.wall_on == "right":
                v = -1
                w = 0.314
                if all(np.array(prox_horizontal[2:]) < 150):
                    self.mode = "wall_following"
                    print(f"{self.mode} on {self.wall_on} activated")
            elif self.wall_on == "left":
                v = -1
                w = - 0.314
                if all(np.array(prox_horizontal[0:3]) < 150):
                    self.mode = "wall_following"
                    print(f"{self.mode} on {self.wall_on} activated")
```

**Step 4**
<br>

Thymio go forward with small curve and when a sensor see the obstacle he continue to go forward but the curve is an other way (go far away from object) until he don’t se the obstacle again. He make this switch all the way of local avoidance
<br>
<div style="text-align: center;">
    <img src="figure/localnav_step4.png" alt="Step4" style="width:70%;"/>
</div><br>
code :
<br>

```py
     elif self.mode == "wall_following":
            if self.wall_on == "right":
                if all(np.array(prox_horizontal[2:]) < 150): # not seeing the wall on right, turning right
                    v, w = 3, -0.25
                else: # seeing the wall, move and turn slightly left
                    v, w = 3, 0.3
            if self.wall_on == "left":
                if all(np.array(prox_horizontal[0:3]) < 150): # not seeing the wall on left, turning left
                    v, w = 3, 0.25
                else: # seeing the wall, move and turn slightly right
                    v, w = 3, -0.3
```


**Step 5**
<br>

Thymio have the point at the beginning(green) and the real position(blue point)
<br>

if the condition under is right so we are back on path. 
$$arctan ((y_{entrance} – y) / (x_{entrance} –x)) = alpha_{entrance}$$
 Change motion_control to get_back to path and take the position (x,y,$alpha$) at the exit
<br>

$alpha$ is the angle of the robot from the x axis
<br>
<div style="text-align: center;">
    <img src="figure/localnav_step5.png" alt="Step5" style="width:70%;"/>
</div>
<br>
code:
<br>

```py
    elif self.control_mode == "local_avoidance":
                get_back_to_path = self.activate_get_back_to_path(x,y, theta)
                if get_back_to_path:
                    print("getting back to path activated")
                    self.control_mode = "get_back_to_path"
                    self.x_exit = x
                    self.y_exit = y
```

```py
    def activate_get_back_to_path(self, x, y, theta):
        dy = y - self.y_entrance
        dx = x - self.x_entrance
        alpha = np.arctan2(dy, dx)
        #d_alpha = (theta - self.alpha_entrance + np.pi) % (2*np.pi) - np.pi
        d_alpha = (alpha - self.alpha_entrance + np.pi) % (2*np.pi) - np.pi
        return ( abs(d_alpha) < 0.1 and np.sqrt(dy**2 + dx**2) > 5 ) # if alpha is small enough, assume that Thymio is back on track
```

**Step 6**
<br>

Thymio go forward at turn dépend on the direction  to go a little bit far away from the object. When he is 4 cm away (enough for the thymio to turn on itself) we change the motion control mode to path_following and recompute the global_path 
<br><br>
<div style="text-align: center;">
    <img src="figure/localnav_step6.png" alt="Step6" style="width:70%;"/>
</div>
<br>
code:
<br>

```py
   elif self.control_mode == "get_back_to_path":
            if self.local_nav.wall_on == "right":
                v, w = 2, 0.314
            else:
                v, w = 2, -0.314
```    

```py
   elif self.control_mode == "get_back_to_path":
            #dtheta = (theta - self.alpha_entrance + np.pi) % (2*np.pi) - np.pi
            distance = np.sqrt(x - self.x_exit, y - self.y_exit)
            if distance > 3: # has moved far enough from exit point
                self.control_mode = "path_following"
                recompute_global_path = True
                print("path following activated activated")
``` 

**Step 7**
<br>

Thymio take the new path and go to the next_target in the dessin the goal 
<br>
<div style="text-align: center;">
    <img src="figure/localnav_step7.png" alt="Step7" style="width:70%;"/>
</div>

## 3. Running the project
Run `main.py`. In particular, depending on laptop, `CAMERA_ID` can be 0 or 1.

First it connects to Thymio and opens a camera window.
include image

Press `m` key to detect the map, obstacles and fix map perspective.
include image

After placing Thymio on the map, press `p` key to find global path and start motion.

**Schematics of main loop**

<div style="text-align: center;">
    <img src="figure/bigschematics.jpg" alt="Schematics" style="width:70%;"/>
</div>



#### 3.1 Explanation of main loop
Main loop roughly consists of 3 parts; map detection, path planning and start motion. Step 1 is run only once in the beginning. There are various flags such as `kidnapping`, `start_motion` and `path_planning` to activate or deactivate step 2 and 3.
```py
    while True:
        if map_detection:
            # Step 1: Detect the map
        if path_planning:
            # Step 2: Path planning
        if start_motion:
            # Step 3: Start motion
``` 

'start_motion' step is the actual control loop, whose steps are roughly as follows.

```py
    if start_motion:
        # get data from camera
        # get data from Thymio
        # check if Thymio is visible on camera
        # check if Thymio is kidnapped
        # update Thymio state using EKF
        # check if next target is reached
        # compute control input and set Thymio target velocity
``` 

`path` variable is a list of nodes to visit, including the goal. `next_target` is the next node to visit. Every time a node is reached, Thymio pops its `next_target` from `path`. If `path` is empty, it means Thymio has reached the goal and the main loop terminates.
```py
    target_reached = np.linalg.norm(np.array([x - next_target[0], y - next_target[1]])) < epsilon
    if target_reached:
        if len(path) == 0:
            print("Goal reached, terminating...")
            break
        else:
            print("Heading towards next waypoint")
            next_target = path.pop(0)
``` 
When Thymio detects kidnapping from acceleration, it sets motor target speed to 0 and activates `path_planning`. 
```py
    # to detect kidnapping
    if abs(acc[2]-22)>4: # sudden change in acc_z indicates kidnapping
        print("kidnapping detected")
        kidnapping = True
        thymio.node.send_set_variables(motors(0,0))
        # computes new global path in the next iteration
        path_planning = True
        continue
``` 
Thymio will start moving only when a new path is computed, and a new path is computed only when kidnapping is over. We use ground sensors to check if kidapping is over, because acceleration sensor cannot differentiate between being on the ground and being held still in the air.
```py
    if path_planning:
        if kidnapping:
            aw(thymio.client.sleep(dt))
            data = thymio.get_data()
            prox_ground_delta = data[2]
            print(f"kidnapping, {prox_ground_delta[0]}")
            if prox_ground_delta[0] > 700: 
                kidnapping = False # kidnapping is over
                print("kidnapping finished")
                continue
        if thymio_found and goal_coords and not kidnapping:
            obstacle_vertices = get_obstacle_vertices(obstacles_contours)
            global_path = compute_global_path(thymio_coords, goal_coords, obstacle_vertices, mask_obstacles)
            # ...
            path = global_path.copy().tolist()
            # global_path is for visualization, and thus in pixel coordinates
            # convert to xy coordinates in centimeters
            path = [convert_pixel_to_cm(target, REAL_MAP_WIDTH_CM, REAL_MAP_HEIGHT_CM, MAP_MAX_WIDTH, MAP_MAX_HEIGHT) for target in path]
            next_target = path.pop(0)
            # once path is computed, start motion and deactivate path planning
            start_motion=True
            path_planning = False
            mc.control_mode = "path_following"
``` 

Motion controller `mc` decides control phase, namely `path_following`, `local_avoidance.turning`, `local_avoidance.wall_following` and `get_back_to_path`. Nominal phase is `path_following`. When a local obstacle is first detected, Thymio will sequentially switch to `local_avoidance.turning`, `local_avoidance.wall_following` and `get_back_to_path` phase. 
``` py
    path_planning = mc.set_mode(prox_horizontal, x, y, theta)
    ul, ur = mc.compute_control(x, y, theta, next_target[0], next_target[1], prox_horizontal)
    thymio.node.send_set_variables(motors(ul, ur))
    aw(thymio.client.sleep(dt))
``` 
Note that during local avoidance, motion controller's primary goal is to avoid the local obstacle, and often moves past the next target node of the original global path. Therefore, when `get_back_to_path` is completed, we activate path planning again.

Finally, when Thymio aruco marker is blocked and EKF is only using speed sensor measurements, state uncertainty will accumulate. To visualize this, we add position uncertainty ellipse, which is obtained from P matrix of `ekf`.
``` py
    xy_variance = ekf.P[0:2, 0:2]
    eigenvalues, eigenvectors = np.linalg.eig(xy_variance)
    ellipse_axis_length = convert_cm_length_to_pixel(np.sqrt(eigenvalues),REAL_MAP_WIDTH_CM, REAL_MAP_HEIGHT_CM, MAP_MAX_WIDTH, MAP_MAX_HEIGHT)
    ellipse_angle = np.arctan2(eigenvectors[0][1], eigenvectors[0][0])
    ellipse_angle = np.rad2deg(-ellipse_angle)
    cv2.ellipse(map_frame, ekf_pixel, ellipse_axis_length, ellipse_angle, 0, 360, (0,255,255), 5)
``` 

#### 3.2 Comments on demo video

Green dot/arrow marks Thymio position and heading from camera. Purple dot/arrow marks Thymio position and heading estimated by EKF.



## 4. Conclusion